# Introduction to StochSS-Compute
  
If you have ever used Gillespy2 to run simulations locally, you should not have too much difficulty running remote simulations, as the syntax is nearly the same. Running a simulation remotely with GillesPy2 requires 3 components: a `Model()` (your data), a `Solver()` (SSA, TauHybrid, ODE, etc.), and a running instance of a `ComputeServer()`. If you do not wish to explicitly state the solver, one will be automatically chosen.

In [15]:
import sys, os
import numpy
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))
import gillespy2
from stochss_compute import RemoteSimulation, ComputeServer

### Creating a GillesPy2 Model  
  
A GillesPy2 model must include species, rates, and reactions.

In [8]:
class MichaelisMenten(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Michaelis_Menten")
            
            ''' 
            PARAMETERS:
            These are constant values relevant to the system, such as reaction kinetic rates.
            
            name: user defined name for reference
            expression: constant value
            '''
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.0017)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.51)
            rate3 = gillespy2.Parameter(name='rate3', expression = 0.11)
            
            # Add parameters to the model
            self.add_parameter([rate1,rate2,rate3])
            
            '''
            SPECIES:
            These can be anything that participates in or is produced by a reaction channel.
            
            name: A user defined name for the species
            initial_value: value/population count of species at start of simulation
            '''
            A = gillespy2.Species(name='A', initial_value=301)
            B = gillespy2.Species(name='B', initial_value=121)
            C = gillespy2.Species(name='C', initial_value=0)
            D = gillespy2.Species(name='D', initial_value=0)
            
            # Add species to the model
            self.add_species([A, B, C, D])
            
            '''Reactions:
            These are the reaction channels causing the system to change over time
            
            name: a user defined name for the reaction
            reactants: dictionary with participant reactants as keys, and consumed per reaction as value.
            products: dictionary with reaction products as keys, and number formed per reaction as value.
            rate: parameter rate constant to be applied to the propensity of this reaction firing
            propensity_function: can be used instead of rate in order to declare a custom propensity function in string format'''
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={C:1},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1,B:1},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1,D:1},
                    rate=rate3)
            
            # Add reactions to the model
            self.add_reaction([r1,r2,r3])
            
            # Set timespan of model
            self.timespan(numpy.linspace(0,100,101))

### Running Simulations and Plotting

First, instantiate a `Model()` and a `ComputeServer()`. After you call `run()`, which returns your future `RemoteResults`, you will have to wait for the simulation to finish. Calling `wait()`, `status()`, `resolve()`, or `cancel()` allows you to interact with your results. Continue reading for more details on the various parameters that you may set.

In [9]:
myModel = MichaelisMenten()

myServer = ComputeServer("localhost", port=1234)

results = RemoteSimulation.on(myServer).with_model(myModel).run()

results = results.resolve()

results.plot()

http://localhost:1234/api/v1/gillespy2/model/run


ValidationError: 1 validation error for ErrorResponse
__root__
  Expecting value: line 1 column 1 (char 0) (type=value_error.jsondecode; msg=Expecting value; doc=<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
  "http://www.w3.org/TR/html4/loose.dtd">
<html>
  <head>
    <title>OSError: Timed out trying to connect to tcp://localhost:8786 after 30 s // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"
        type="text/css">
    <!-- We need to make sure this has a favicon so that the debugger does
         not accidentally trigger a request to /favicon.ico which might
         change the application's state. -->
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script type="text/javascript">
      var TRACEBACK = 140359511121344,
          CONSOLE_MODE = false,
          EVALEX

While the model.run() function can be called without any given arguments, GillesPy2 provides several options for customizing your simulations. The following keyword arguments can be used in the model.run() function to customize your simulations:
<br/><br/>
### model.run() kwargs
**solver=[solver]**  
    manually choose a solver/algorithm one of the following GillesPy2 solvers:  
    [ODESolver()](./StartingModels/Michaelis-Menten_Basic_ODE.ipynb)  
    [SSASolver()](./StartingModels/Michaelis-Menten_NumPy_SSA.ipynb)    
    [SSACSolver()](./StartingModels/Michaelis-Menten_SSA_C.ipynb)    
    [TauLeapingSolver()](./StartingModels/Michaelis-Menten_Basic_Tau_Leaping.ipynb)    
    [TauHybridSolver()](./StartingModels/Michaelis-Menten_Basic_Tau_Hybrid.ipynb)    
    
**number_of_trajectories=1**  
    [int]: Number of times to run the current simulation  
    
**seed=None**  
    [int]: Define a custom seed for random number generation

**show_labels=True**  
    [bool]: When set to true (default) returns results in a dictonary with time and species as dictionary keys, and relevant trajectory vectors as value pair.  When set to False, results will be returned as a numpy array with dimensions [timestep][species_index].  Either way, the the results of each trajectory are stored in a list.

<br/><br/>
### results.plot() kwargs
***xaxis_label='Time (s)'***  
    [string]: Label for the plot's x-axis  
***yaxis_label='Species Population'***  
    [string]: Label for the plot's y-axis  
***style='default'***  
    [string]: matplotlib stylesheets https://matplotlib.org/3.1.1/gallery/style_sheets/style_sheets_reference.html  
***title=None***  
    [string]: title of figure  
***show_legend=True***  
    [bool]: Show legend if true.  
***multiple_graphs=False***  
    [bool]: if each trajectory should have its own graph or if they should overlap  
***included_species_list=[]***  
    [list]: A list of strings describing which species to include by species name. By default displays all species.  
    
<br/><br/>
GillesPy2 also offers built-in offline plotly plotting and statistical data plotting.  [See the documents for more details.](https://gillespy2.readthedocs.io)  
<br/><br/>
  
### Solver specific kwargs
**ODESolver, TauHybridSolver: integrator='lsoda'**  
    [String]:  
integrator to be used form scipy.integrate.ode. Options include 'vode', 'zvode', 'lsoda', 'dopri5', and 'dop835'.  For more details, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.ode.html  
  
**ODESolver, TauHybridSolver: integrator_options={}**  
    [dictionary]:  
contains options to the scipy integrator. for a list of options, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.ode.html  
  
**TauLeapingSolver, TauHybridSolver: tau_tol=0.03**  
    [float]:  
Relative error tolerance value for calculating tau_step. value should be between 0.0-1.0  
  
**TauHybridSolver: switch_tol=0.03**  
    [float]:  
Relative error tolerance value for switching between deterministic/stochastic.  value should be between 0.0-1.0 
<br/><br/>

## HYBRID - ADVANCED

Some of the solvers in GillesPy2 provide further options for customizing simulations.  For examples on how to use these advanced features, please see the following links:  

[Hybrid: Using Rate Rules to model continuous species](./AdvancedExamples/Hybrid/hybrid_continuous_species.ipynb)  
[Hybrid: Dynamically switching between stochastic and deterministic representations](./AdvancedExamples/Hybrid/hybrid_switching_example.ipynb)  


